In [1]:
import sympy
from sympy import Symbol

from mlscorecheck.core import (score_function_set, sqrt)

In [2]:
tp = Symbol('tp')
fp = Symbol('fp')
tn = Symbol('tn')
fn = Symbol('fn')
eps = Symbol('eps')
p = Symbol('p')
n = Symbol('n')
beta = Symbol('beta')

In [3]:
atomic_symbols = {'tp': tp, 
                  'fp': fp, 
                  'tn': tn, 
                  'fn': fn, 
                  'eps': eps, 
                  'p': p, 
                  'n': n,
                  'beta': beta}

In [4]:
score_functions = score_function_set()
#del score_functions['mcc']

In [5]:
score_functions

{'acc': <function mlscorecheck.core._scores.accuracy(*, tp, tn, p, n)>,
 'sens': <function mlscorecheck.core._scores.sensitivity(*, tp, p)>,
 'spec': <function mlscorecheck.core._scores.specificity(*, tn, n)>,
 'npv': <function mlscorecheck.core._scores.negative_predictive_value(*, tn, fn)>,
 'ppv': <function mlscorecheck.core._scores.positive_predictive_value(*, tp, fp)>,
 'f1_plus': <function mlscorecheck.core._scores.f1_plus(*, tp, fp, p)>,
 'f1_minus': <function mlscorecheck.core._scores.f1_minus(*, tn, fn, n)>,
 'fm': <function mlscorecheck.core._scores.fowlkes_mallows_index(*, tp, fp, p)>,
 'bm': <function mlscorecheck.core._scores.informedness(*, tp, tn, p, n)>,
 'pt': <function mlscorecheck.core._scores.prevalence_threshold(*, tp, fp, p, n)>,
 'lrp': <function mlscorecheck.core._scores.positive_likelihood_ratio(*, tp, fp, p, n)>,
 'lrn': <function mlscorecheck.core._scores.negative_likelihood_ratio(*, tn, fn, p, n)>,
 'mk': <function mlscorecheck.core._scores.markedness(*, tp, 

In [6]:
score_symbols = {key: Symbol(key) for key in score_functions}

In [7]:
score_symbols

{'acc': acc,
 'sens': sens,
 'spec': spec,
 'npv': npv,
 'ppv': ppv,
 'f1_plus': f1_plus,
 'f1_minus': f1_minus,
 'fm': fm,
 'bm': bm,
 'pt': pt,
 'lrp': lrp,
 'lrn': lrn,
 'mk': mk,
 'dor': dor,
 'mcc': mcc,
 'ji': ji,
 'ba': ba,
 'kappa': kappa,
 'gm': gm,
 'upm': upm}

In [8]:
score_equations = {}

for key in score_symbols:
    function = score_functions[key]
    params = function.__code__.co_varnames[:function.__code__.co_kwonlyargcount]
    params = {param: atomic_symbols[param] for param in params}
    score_equations[key] = score_symbols[key] - score_functions[key](**params)

In [9]:
score_equations

{'acc': acc - (tn + tp)/(n + p),
 'sens': sens - tp/p,
 'spec': spec - tn/n,
 'npv': npv - tn/(fn + tn),
 'ppv': ppv - tp/(fp + tp),
 'f1_plus': f1_plus - 2*tp/(fp + p + tp),
 'f1_minus': f1_minus - 2*tn/(fn + n + tn),
 'fm': fm - sqrt(tp**2/(p*(fp + tp))),
 'bm': bm + 1 - tp/p - tn/n,
 'pt': pt - (-fp/n + sqrt(fp*tp/(n*p)))/(-fp/n + tp/p),
 'lrp': lrp - n*tp/(fp*p),
 'lrn': -fn*n/(p*tn) + lrn,
 'mk': mk - tn/(fn + tn) - tp/(fp + tp) + 1,
 'dor': dor - tn*tp/(fn*fp),
 'mcc': mcc - (-fn*fp + tn*tp)/sqrt(n*p*(fn + tn)*(fp + tp)),
 'ji': ji - tp/(fp + p),
 'ba': ba - 0.5*tp/p - 0.5*tn/n,
 'kappa': kappa - (tn + tp)/((n + p)*((tn + tp)*(n + p - tn - tp)/(2*tn*tp - 2*(n - tp)*(p - tn)) + (tn + tp)/(n + p))),
 'gm': gm - sqrt(tn*tp/(n*p)),
 'upm': -8*tn*tp/((2*tn/(n + p + tn - tp) + 2*tp/(n + p - tn + tp))*(n + p - tn + tp)*(n + p + tn - tp)) + upm}

In [10]:
tp = Symbol('tp')
fp = Symbol('fp')
tn = Symbol('tn')
fn = Symbol('fn')
eps = Symbol('eps')
p = Symbol('p')
n = Symbol('n')
beta = Symbol('beta')

In [11]:
score_symbols = {key: Symbol(key) for key in score_functions}

In [12]:
score_equations = {}

for key in score_symbols:
    function = score_functions[key]
    params = function.__code__.co_varnames[:function.__code__.co_kwonlyargcount]
    params = {param: atomic_symbols[param] for param in params}
    score_equations[key] = score_symbols[key] - score_functions[key](**params)

In [13]:
scores = list(score_equations.keys())

for idx, score0 in enumerate(scores):
    for score1 in scores[idx+1:]:
        if (score0 == 'fm' and score1 in ('mk', 'gm', 'upm')) \
            or (score0 == 'pt' and score1 in ('upm'))\
            or (score0 == 'mk' and score1 in ('kappa', 'gm', 'upm'))\
            or (score0 == 'dor' and score1 in ('kappa', 'upm'))\
            or (score0 == 'mcc' and score1 in ('gm', 'upm'))\
            or (score0 == 'kappa' and score1 in ('upm')):
            continue
        print(score0, score1)
        results = sympy.solve([score_equations[score0], 
                               score_equations[score1], 
                               p-tp-fn, 
                               n-tn-fp], 
                              [tp, tn, fp, fn], 
                              dict=True,
                              check=False)
        print(len(results), results)

acc sens
1 [{tp: p*sens, tn: acc*n + acc*p - p*sens, fp: -acc*n - acc*p + n + p*sens, fn: -p*sens + p}]
acc spec
1 [{tp: acc*n + acc*p - n*spec, tn: n*spec, fp: -n*spec + n, fn: -acc*n - acc*p + n*spec + p}]
acc npv
1 [{tp: (acc*n*npv - acc*n + acc*npv*p - acc*p + npv*p)/(2*npv - 1), tn: (acc*n*npv + acc*npv*p - npv*p)/(2*npv - 1), fp: (-acc*n*npv - acc*npv*p + 2*n*npv - n + npv*p)/(2*npv - 1), fn: (-acc*n*npv + acc*n - acc*npv*p + acc*p + npv*p - p)/(2*npv - 1)}]
acc ppv
1 [{tp: (acc*n*ppv + acc*p*ppv - n*ppv)/(2*ppv - 1), tn: (acc*n*ppv - acc*n + acc*p*ppv - acc*p + n*ppv)/(2*ppv - 1), fp: (-acc*n*ppv + acc*n - acc*p*ppv + acc*p + n*ppv - n)/(2*ppv - 1), fn: (-acc*n*ppv - acc*p*ppv + n*ppv + 2*p*ppv - p)/(2*ppv - 1)}]
acc f1_plus
1 [{tp: (acc*f1_plus*n + acc*f1_plus*p - f1_plus*n - f1_plus*p)/(2*f1_plus - 2), tn: (acc*f1_plus*n + acc*f1_plus*p - 2*acc*n - 2*acc*p + f1_plus*n + f1_plus*p)/(2*f1_plus - 2), fp: (-acc*f1_plus*n - acc*f1_plus*p + 2*acc*n + 2*acc*p + f1_plus*n - f1_plus*p 